In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score

In [2]:
# Load dataset
file_path = '/content/ObesityDataSet_raw_and_data_sinthetic.csv'
csv = pd.read_csv(file_path)

In [4]:
label_encoder = LabelEncoder()
#Assign the csv dataframe to data
data = csv
data['NObeyesdad'] = label_encoder.fit_transform(data['NObeyesdad'])  # Encode kolom target 'NObeyesdad'

# Identifikasi kolom kategorikal dan lakukan encoding
kolom_kategorikal = data.select_dtypes(include=['object']).columns  # Menemukan kolom dengan tipe data 'object'
for kolom in kolom_kategorikal:
    if kolom != 'NObeyesdad':  # Jangan encode kolom target
        data[kolom] = LabelEncoder().fit_transform(data[kolom])  # Lakukan encoding pada kolom kategorikal

# Pisahkan fitur dan target
X = data.drop(columns=['NObeyesdad'])
y = data['NObeyesdad']

In [5]:
# Isi nilai yang hilang dan normalisasi fitur
X = X.fillna(0)  # Ganti nilai yang hilang dengan 0
scaler = StandardScaler()  # Inisialisasi objek StandardScaler untuk normalisasi fitur
X = scaler.fit_transform(X)  # Lakukan normalisasi pada fitur

# Bagi data menjadi set pelatihan dan pengujian (80% pelatihan, 20% pengujian)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Konversi data pelatihan dan pengujian ke tensor PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)  # Data pelatihan sebagai tensor
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)  # Data pengujian sebagai tensor
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)  # Label pelatihan sebagai tensor
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)  # Label pengujian sebagai tensor

# Pindahkan tensor ke GPU jika tersedia
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Tentukan perangkat (GPU jika tersedia, CPU jika tidak)
X_train_tensor = X_train_tensor.to(device)  # Pindahkan data pelatihan ke perangkat yang dipilih
X_test_tensor = X_test_tensor.to(device)  # Pindahkan data pengujian ke perangkat yang dipilih
y_train_tensor = y_train_tensor.to(device)  # Pindahkan label pelatihan ke perangkat yang dipilih
y_test_tensor = y_test_tensor.to(device)  # Pindahkan label pengujian ke perangkat yang dipilih

In [6]:
# Model Bidirectional RNN dengan pooling
class ModelBidirectionalRNN(nn.Module):
    def __init__(self, ukuran_input, ukuran_hidden, jumlah_kelas, jenis_pooling):
        super(ModelBidirectionalRNN, self).__init__()

        # Bidirectional RNN: Menggunakan dua arah (forward dan backward)
        self.rnn = nn.RNN(ukuran_input, ukuran_hidden, batch_first=True, bidirectional=True)

        # Jenis pooling yang dipilih: 'max' atau 'avg'
        self.jenis_pooling = jenis_pooling

        # Lapisan fully connected untuk klasifikasi, ukuran hidden dilipat dua karena bidirectional
        self.fc = nn.Linear(ukuran_hidden * 2, jumlah_kelas)

    def forward(self, x):
        # Forward pass melalui Bidirectional RNN
        out, _ = self.rnn(x)

        # Pilih metode pooling: 'max' atau 'avg'
        if self.jenis_pooling == 'max':
            out = torch.max(out, dim=1).values  # Pooling maksimum (mengambil nilai maksimal)
        elif self.jenis_pooling == 'avg':
            out = torch.mean(out, dim=1)  # Pooling rata-rata (mengambil rata-rata dari output)

        # Lapisan klasifikasi akhir untuk prediksi kelas
        out = self.fc(out)
        return out

In [7]:
# Hyperparameter eksperimen
ukuran_input = X_train_tensor.shape[1]  # Menentukan jumlah fitur input berdasarkan data pelatihan
jumlah_kelas = len(label_encoder.classes_)  # Menentukan jumlah kelas output berdasarkan label encoder

# Daftar ukuran hidden layer yang akan diuji
ukuran_hidden_list = [32, 64, 128]

# Daftar metode pooling yang akan diuji: 'max' atau 'avg'
jenis_pooling_list = ['max', 'avg']

# Daftar jumlah epoch yang akan diuji: 5, 50, 100, 250, 350
daftar_epoch = [5, 50, 100, 250, 350]

# Daftar optimizer yang akan diuji: SGD, RMSprop, dan Adam
daftar_optimizer = ['SGD', 'RMSprop', 'Adam']

# Variabel untuk menyimpan model terbaik
model_terbaik = None  # Untuk menyimpan model dengan akurasi terbaik
akurasi_terbaik = 0  # Menyimpan nilai akurasi terbaik
parameter_terbaik = {}  # Menyimpan parameter hyperparameter terbaik yang digunakan
ringkasan_hasil = []  # Menyimpan ringkasan hasil dari eksperimen

In [8]:
# Eksperimen kombinasi hyperparameter
for ukuran_hidden in ukuran_hidden_list:  # Iterasi ukuran hidden layer
    for jenis_pooling in jenis_pooling_list:  # Iterasi metode pooling ('max' atau 'avg')
        for nama_optimizer in daftar_optimizer:  # Iterasi jenis optimizer ('SGD', 'RMSprop', atau 'Adam')
            for epoch in daftar_epoch:  # Iterasi jumlah epoch yang akan diuji
                print(f"--- Ukuran Hidden: {ukuran_hidden}, Pooling: {jenis_pooling}, Optimizer: {nama_optimizer}, Epoch: {epoch} ---")

                # Inisialisasi model dan fungsi loss
                model = ModelBidirectionalRNN(ukuran_input, ukuran_hidden, jumlah_kelas, jenis_pooling).to(device)  # Inisialisasi model
                fungsi_loss = nn.CrossEntropyLoss()  # Fungsi loss untuk klasifikasi

                # Pilih optimizer berdasarkan pilihan yang ada
                if nama_optimizer == 'SGD':
                    optimizer = optim.SGD(model.parameters(), lr=0.01)  # Optimizer SGD
                elif nama_optimizer == 'RMSprop':
                    optimizer = optim.RMSprop(model.parameters(), lr=0.01)  # Optimizer RMSprop
                elif nama_optimizer == 'Adam':
                    optimizer = optim.Adam(model.parameters(), lr=0.01)  # Optimizer Adam

                # Pengaturan Early Stopping untuk menghindari overfitting
                batas_sabar = 10  # Batas toleransi untuk tidak ada peningkatan (early stopping)
                loss_terbaik = float('inf')  # Nilai awal loss terbaik
                penghitung_sabar = 0  # Penghitung sabar untuk early stopping

                # Loop pelatihan untuk setiap epoch
                for ep in range(epoch):  # Iterasi berdasarkan jumlah epoch
                    model.train()  # Set model ke mode pelatihan
                    optimizer.zero_grad()  # Bersihkan gradien dari iterasi sebelumnya
                    output = model(X_train_tensor.unsqueeze(1))  # Forward pass melalui model
                    loss = fungsi_loss(output, y_train_tensor)  # Hitung loss
                    loss.backward()  # Backpropagation untuk menghitung gradien
                    optimizer.step()  # Update parameter model

                    # Validasi untuk memantau performa model pada data uji
                    model.eval()  # Set model ke mode evaluasi
                    with torch.no_grad():  # Tidak menghitung gradien selama validasi
                        val_output = model(X_test_tensor.unsqueeze(1))  # Forward pass untuk validasi
                        val_loss = fungsi_loss(val_output, y_test_tensor).item()  # Hitung loss pada data validasi
                        prediksi_val = val_output.argmax(dim=1)  # Ambil kelas dengan probabilitas tertinggi
                        akurasi_val = accuracy_score(y_test_tensor.cpu(), prediksi_val.cpu()) * 100  # Hitung akurasi

                    # Log detail setiap epoch
                    print(f"Epoch {ep + 1}/{epoch}, Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}, Akurasi: {akurasi_val:.2f}%")

                    # Cek early stopping berdasarkan validasi loss
                    if val_loss < loss_terbaik:
                        loss_terbaik = val_loss  # Simpan loss terbaik
                        penghitung_sabar = 0  # Reset penghitung sabar
                    else:
                        penghitung_sabar += 1  # Tambah penghitung sabar

                    # Jika penghitung sabar melebihi batas, hentikan pelatihan lebih awal
                    if penghitung_sabar >= batas_sabar:
                        print("Early stopping diaktifkan.")
                        break

                # Evaluasi model setelah pelatihan
                model.eval()  # Set model ke mode evaluasi
                with torch.no_grad():  # Tidak menghitung gradien selama evaluasi
                    prediksi = model(X_test_tensor.unsqueeze(1)).argmax(dim=1)  # Prediksi pada data uji
                    akurasi = accuracy_score(y_test_tensor.cpu(), prediksi.cpu())  # Hitung akurasi

                # Log akurasi akhir untuk kombinasi hyperparameter saat ini
                print(f"Akurasi Akhir untuk Ukuran Hidden: {ukuran_hidden}, Pooling: {jenis_pooling}, Optimizer: {nama_optimizer}, Epoch: {epoch}: {akurasi:.6f}")
                ringkasan_hasil.append(f"{ukuran_hidden},{jenis_pooling},{nama_optimizer},{epoch},{akurasi}")  # Simpan hasil eksperimen

                # Simpan model dan parameter terbaik jika akurasi lebih baik
                if akurasi > akurasi_terbaik:
                    akurasi_terbaik = akurasi  # Update akurasi terbaik
                    model_terbaik = model  # Simpan model terbaik
                    parameter_terbaik = {  # Simpan parameter terbaik
                        'ukuran_hidden': ukuran_hidden,
                        'jenis_pooling': jenis_pooling,
                        'optimizer': nama_optimizer,
                        'epoch': epoch
                    }

Streaming output truncated to the last 5000 lines.
Epoch 5/250, Loss: 1.9948, Val Loss: 1.9805, Akurasi: 14.18%
Epoch 6/250, Loss: 1.9909, Val Loss: 1.9767, Akurasi: 14.89%
Epoch 7/250, Loss: 1.9870, Val Loss: 1.9728, Akurasi: 15.13%
Epoch 8/250, Loss: 1.9831, Val Loss: 1.9690, Akurasi: 15.37%
Epoch 9/250, Loss: 1.9792, Val Loss: 1.9652, Akurasi: 16.08%
Epoch 10/250, Loss: 1.9754, Val Loss: 1.9615, Akurasi: 17.02%
Epoch 11/250, Loss: 1.9716, Val Loss: 1.9577, Akurasi: 18.20%
Epoch 12/250, Loss: 1.9678, Val Loss: 1.9540, Akurasi: 18.44%
Epoch 13/250, Loss: 1.9640, Val Loss: 1.9503, Akurasi: 19.15%
Epoch 14/250, Loss: 1.9603, Val Loss: 1.9466, Akurasi: 19.86%
Epoch 15/250, Loss: 1.9565, Val Loss: 1.9429, Akurasi: 21.04%
Epoch 16/250, Loss: 1.9528, Val Loss: 1.9393, Akurasi: 22.46%
Epoch 17/250, Loss: 1.9491, Val Loss: 1.9356, Akurasi: 23.40%
Epoch 18/250, Loss: 1.9454, Val Loss: 1.9320, Akurasi: 25.30%
Epoch 19/250, Loss: 1.9418, Val Loss: 1.9284, Akurasi: 25.53%
Epoch 20/250, Loss: 1.93